In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd '/content/drive/MyDrive/THESIS/rag-thesis'

/content/drive/MyDrive/THESIS/rag-thesis


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install -U -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11; 1.6.2 Requires-Python >=3.7,<3.10; 1.6.3 Requires-Python >=3.7,<3.10; 1.7.0 Requires-Python >=3.7,<3.10; 1.7.1 Requires-Python >=3.7,<3.10; 1.7.2 Requires-Python >=3.7,<3.11; 1.7.3 Requires-Python >=3.7,<3.11; 1.8.0 Requires-Python >=3.8,<3.11; 1.8.0rc1 Requires-Python >=3.8,<3.11; 1.8.0rc2 Requires-Python >=3.8,<3.11; 1.8.0rc3 Requires-Python >=3.8,<3.11; 1.8.0rc4 Requires-Python >=3.8,<3.11; 1.8.1 Requires-Python >=3.8,<3.11
ERROR: Could not find a version that satisfies the requirement tdqdm (

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import pymc as pm
import arviz as az
from forecasting_utils import *

In [6]:
df_processed = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv')
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
df_processed.head()

,observation_date,GDP_log,XTIMVA01USM664S_log,XTEXVA01USM664S_log,GPDI_log,RPI_log,INDPRO_log,BUSLOANS_log,NONREVSL_log,UNRATE_diff,...,GS1_diff,AAA_diff,IRLTLT01USM156N_diff,M1V,CUMFNS,CLF16OV,B235RC1Q027SBEA,ROWFDIQ027S,CPILFESL_inflation,PCEPI_inflation
0,1960-04-01,-1.889106,-1.937830,-1.900735,-1.933337,-1.969102,-2.223121,-1.891318,-1.855541,0.143477,...,-0.991366,-0.273408,-0.492671,-1.208557,0.410447,-1.730502,-0.917074,-0.751348,-1.084373,-0.461731
1,1960-07-01,-1.882088,-1.966669,-1.884644,-1.940230,-1.964744,-2.260716,-1.847751,-1.841514,0.420758,...,-1.133084,-0.381349,-0.925014,-1.210446,-0.080174,-1.724082,-0.920047,-0.750873,-1.585535,-0.688792
2,1960-10-01,-1.890482,-2.003349,-1.890457,-2.047059,-1.966695,-2.316039,-1.805262,-1.830178,1.113960,...,-0.112715,0.023429,0.134228,-1.227453,-0.698059,-1.713532,-0.921271,-0.752034,-0.573631,-0.579965
3,1961-01-01,-1.882994,-2.008331,-1.877366,-2.021036,-1.951916,-2.350629,-1.773634,-1.821522,0.698039,...,-0.169402,-0.138482,-0.211647,-1.220367,-1.108528,-1.704369,-0.921796,-0.750767,-1.086546,-0.984947
4,1961-04-01,-1.866870,-2.003468,-1.911936,-1.962365,-1.926400,-2.262478,-1.750164,-1.825352,0.282117,...,0.099862,0.023429,0.004525,-1.198636,-0.596247,-1.704078,-0.922670,-0.750318,-0.916503,-1.296390


### Forecasting Benchmark Implementation:

To conduct the sampling of RMSEs for each variable under each forecast horizion for each model, we will do the following:

- Starting with forecast h=1 and a a 70-30 training-test split, we will iterate over each variable and employ forecasting with horizion h. In other words, starting at quarter i into the time series data, we will forcast the next h quarters. At the time time, we use a univariate AR(1) model to forecast the same next h quarters.

- We will then calculate the MSE beteween the actual and forecasted values for each variable. We do the same between the AR(1) predicted and actual values. This is one obvervation.

- We then forecast the next h quarters starting at quarter i+1 into the time series and calculate the MSE of the forecast for each variable relative to the AR(1). This is the second observation. in other words, we have shifted 1 quarter out of the test set and into the training set.

- We repeat this process until 95% of the data is in the training set.

- From these obvervations, we sum up all mse's for the model, sum up all the mses of the AR(1), and then we take the square root of the mdoel forecasting error and the AR(1) error. This is the RMSFE. We then take the mean, std, min, max, q25, q75, and iqr. These are the summary statistics for the models RMSFE relative to AR(1) with horizion h.

- We then repeat the process for h=3, h=6, and h=12.

- We then repeat the process for each model.

**For TimeGPT, we will shift the training-test split by 2% of the data each iteration due to API limits

### AR(1) Benchmark

In [10]:
# Example usage:
target_variable = 'GDP_log'
# Try different AR orders
train_data, test_data, _ = test_train_split(df_processed, test_size=0.3)
print(run_ar_benchmark(train_data, test_data, target_variable, ar_order=1, horizon=10))

(2.254774871138849e-05, np.float64(0.44906531416485146))


In [ ]:
df_processed = df_processed.select_dtypes(include=[np.number])
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
ar1_results = {
    h:{
        'rmses': {var:[] for var in variables},
    } for h in ["h=1", "h=3", "h=6", "h=12"]
}

for h in [1,3,6,12]:
    print("Forecasting on horizon: " + str(h))
    test_size = 0.3
    train_size = int(len(df_processed)*(1-test_size))
    train_data = df_processed.iloc[:train_size]
    test_data = df_processed.iloc[train_size:]

    while len(test_data) > 13:

        for var in variables:
            ar_rmse, _ = run_ar_benchmark(train_data, test_data, var, ar_order=1, horizon=h)
            ar1_results["h="+str(h)]['rmses'][var].append(ar_rmse)

        train_size += 1
        train_data = df_processed.iloc[:train_size]
        test_data = df_processed.iloc[train_size:]


print(ar1_results)

Forecasting on horizon: 1
Forecasting on horizon: 3
Forecasting on horizon: 6
Forecasting on horizon: 12
{'h=1': {'rmses': {'GDP_log': [np.float64(0.0019525434927206708), np.float64(0.0021372210135013336), np.float64(0.0005326093475404026), np.float64(0.005154037350663998), np.float64(0.0026082653258706623), np.float64(0.0049230400324511425), np.float64(0.0017163740524098747), np.float64(0.0013877638216753807), np.float64(0.0008640492740221717), np.float64(0.0023761425896761112), np.float64(0.003053464495645364), np.float64(0.012159614496598192), np.float64(0.0023954433831894217), np.float64(0.00952034560326842), np.float64(0.027857197154308455), np.float64(0.02113198051802545), np.float64(0.013444735663079466), np.float64(0.00638770488781959), np.float64(0.001487601128308702), np.float64(0.003724389908185688), np.float64(0.002072347801175045), np.float64(0.0011043178201368686), np.float64(0.0008003318539280269), np.float64(0.0077444316899717425), np.float64(0.001307188130751702), np.f

In [ ]:
# Save to JSON
import json
with open('results/ar1_results.json', 'w') as f:
    json.dump(ar1_results, f,indent=4,)

### Bayesian Vector Autoregression (BVAR)

In [ ]:
df_processed = df_processed.select_dtypes(include=[np.number])
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
bvar_results = {
    h:{
        'RMSE': [],
        'cum_RMSE': {var:[] for var in variables}
    } for h in ["h=1", "h=3", "h=6", "h=12"]
}

for h in [1,3,6,12]:
    print("Forecasting on horizon: " + str(h))
    test_size = 0.3
    train_size = int(len(df_processed)*(1-test_size))
    train_data = df_processed.iloc[:train_size]
    test_data = df_processed.iloc[train_size:]

    current_cum_sums ={var:0 for var in variables}
    rmses = {var:[] for var in variables}
    ar_rmses = {var:[] for var in variables}

    while len(test_data) > 13:

        rmse_dict, _ = fit_evaluate_bvar_pymc(train_data, test_data, horizon=h, lag_order=2, num_samples=200)
        for var in variables:
            ar_rmse, _ = run_ar_benchmark(train_data, test_data, var, ar_order=1, horizon=h)
            current_cum_sums[var] += rmse_dict[var]
            bvar_results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
            rmses[var].append(rmse_dict[var])
            ar_rmses[var].append(ar_rmse)


        train_size += 3
        train_data = df_processed.iloc[:train_size]
        test_data = df_processed.iloc[train_size:]

    for var in variables:
        bvar_results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))

print(bvar_results)

Forecasting on horizon: 1


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


...

Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


{'h=1': {'RMSE': [np.float64(1.2575354631550404), np.float64(1.2612386065549617), np.float64(1.1888927532108888), np.float64(0.6225513511468176), np.float64(1.0846246319763455), np.float64(0.9779041316592991), np.float64(0.7353147127409809), np.float64(1.8534342411894842), np.float64(1.0759567504445189), np.float64(2.6088922475737304), np.float64(2.0581741275815815), np.float64(0.9373157822684454), np.float64(0.930420787341747), np.float64(1.0322147007203102), np.float64(0.9062108256758649), np.float64(0.8896721316536296), np.float64(1.0663099052481277), np.float64(0.9531274573934597), np.float64(1.3307927812731002), np.float64(1.0129550056747967)], 'cum_RMSE': {'GDP_log': [np.float64(0.009463726907510517), np.float64(0.021959197409233244), np.float64(0.030646336919616712), np.float64(0.032915877027237306), np.float64(0.03377293691507621), np.float64(0.05123412411323047), np.float64(0.057513689744956076), np.float64(0.06321357241661607), np.float64(0.06739958101493149), np.float64(0.07

In [ ]:
for h in bvar_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(bvar_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 8]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.1891769197241566
Std:  0.46173676079097314
Min:  0.6225513511468176
Max:  2.6088922475737304
Q25:  0.9355920335367708
Median:  1.049262302984219
Q75:  1.2584612490050207
IQR:  0.32286921546824987
Horizon h=3:
Mean: 1.3627662228883985
Std:  0.39222049709816686
Min:  0.9464924964879319
Max:  2.5429157116639773
Q25:  1.0860690325833773
Median:  1.2016576425814443
Q75:  1.5456999271976994
IQR:  0.45963089461432216
Horizon h=6:
Mean: 1.4808805902972555
Std:  0.4270060395687694
Min:  0.9162185272700611
Max:  2.4289995964195916
Q25:  1.1786163786115307
Median:  1.3196961764429542
Q75:  1.7176546849141885
IQR:  0.5390383063026578
Horizon h=12:
Mean: 1.6559881174074094
Std:  0.6722376110400714
Min:  0.8255794278234918
Max:  3.1342980759618557
Q25:  1.1775318769100738
Median:  1.4710219297358091
Q75:  1.9109255705583714
IQR:  0.7333936936482977


In [ ]:
# Save to JSON
import json
with open('results/bvar_results.json', 'w') as f:
    json.dump(bvar_results, f,indent=4,)

### ARIMA

In [ ]:
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
arima_results = {
    h:{
        'RMSE': [],
        'cum_RMSE': {var:[] for var in variables}
    } for h in ["h=1", "h=3", "h=6", "h=12"]
}

for h in [1,3,6,12]:
    print("Forecasting on horizon: " + str(h))
    test_size = 0.3
    train_size = int(len(df_processed)*(1-test_size))
    train_data = df_processed.iloc[:train_size]
    test_data = df_processed.iloc[train_size:]

    current_cum_sums ={var:0 for var in variables}
    rmses = {var:[] for var in variables}
    ar_rmses = {var:[] for var in variables}

    while len(test_data) > 13:
        for var in variables:

            rmse, _ = fit_evaluate_arima(var, train_data, test_data, h)
            ar_rmse, _ = run_ar_benchmark(train_data, test_data, var, ar_order=1, horizon=h)

            current_cum_sums[var] += rmse
            #print(current_cum_sums[var])
            arima_results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
            rmses[var].append(rmse)
            ar_rmses[var].append(ar_rmse)

        train_size += 1
        train_data = df_processed.iloc[:train_size]
        test_data = df_processed.iloc[train_size:]


    for var in variables:
        arima_results["h="+str(h)]['RMSE'].append(np.sum(rmses[var])/np.sum(ar_rmses[var]))



Forecasting on horizon: 1
Forecasting on horizon: 3
Forecasting on horizon: 6
Forecasting on horizon: 12


In [ ]:
for h in arima_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(arima_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 5]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

NameError: name 'arima_results' is not defined

In [ ]:
# Save to JSON
import json
with open('results/arima_results.json', 'w') as f:
    json.dump(arima_results, f,indent=4,)

### TimeGPT

Multiseries Forecast:
https://docs.nixtla.io/docs/capabilities-forecast-multiple_series_forecasting



In [11]:
pip install -Uqq nixtla datasetsforecast utilsforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00


In [12]:
from nixtla import NixtlaClient
from datasetsforecast.long_horizon import LongHorizon
from utilsforecast.losses import mae

nixtla_client = NixtlaClient(
    # defaults to os.environ.get("NIXTLA_API_KEY")
    api_key = 'nixak-Q6bqxtnGbhVX0aqgWPvnwwnAIeyRk4dSJJELRDUBnoYwYnsaUBh1rHTpzhgdX5Bdp9wUCmpyB83SYhIo'
)

In [13]:
# Prepare data for TimeGPT by reshaping from wide to long format
def prepare_data_for_timegpt(df):
    # Create a copy to avoid modifying the original
    df = df.copy()

    # Rename the date column to 'ds'
    df = df.rename(columns={'observation_date': 'ds'})

    # Get list of numeric columns (these will be our variables to forecast)
    numeric_columns = df.select_dtypes(include=[np.number]).columns

    # Melt the dataframe to convert from wide to long format
    df_long = pd.melt(
        df,
        id_vars=['ds'],
        value_vars=numeric_columns,
        var_name='unique_id',
        value_name='y'
    )

    # Sort by unique_id and date
    df_long = df_long.sort_values(['unique_id', 'ds'])

    return df_long

# Apply the transformation
df_processed = pd.read_csv('/content/drive/MyDrive/THESIS/rag-thesis/data/processed_FRED_data.csv')
variables = list(df_processed.select_dtypes(include=[np.number]).columns)
df_timegpt = prepare_data_for_timegpt(df_processed)

# Display the first few rows of the transformed data
print("Shape of transformed data:", df_timegpt.shape)
print("\nFirst few rows of transformed data:")
print(df_timegpt.head())

# Display unique variables being forecast
print("\nUnique variables being forecast:")
print(df_timegpt['unique_id'].unique())

Shape of transformed data: (5160, 3)

First few rows of transformed data:
              ds unique_id         y
2838  1960-04-01  AAA_diff -0.273408
2839  1960-07-01  AAA_diff -0.381349
2840  1960-10-01  AAA_diff  0.023429
2841  1961-01-01  AAA_diff -0.138482
2842  1961-04-01  AAA_diff  0.023429

Unique variables being forecast:
['AAA_diff' 'B235RC1Q027SBEA' 'BUSLOANS_log' 'CLF16OV'
 'CPILFESL_inflation' 'CUMFNS' 'DFF_diff' 'GDP_log' 'GPDI_log' 'GS1_diff'
 'INDPRO_log' 'IRLTLT01USM156N_diff' 'M1V' 'NONREVSL_log'
 'PCEPI_inflation' 'ROWFDIQ027S' 'RPI_log' 'UNRATE_diff'
 'XTEXVA01USM664S_log' 'XTIMVA01USM664S_log']


In [ ]:
def run_timegpt(df_long, df_transformed, ratio=0.7):
    # Horizons to evaluate
    horizons = [1, 3, 6, 12]
    results = {
        h: {
            'RMSE': [],
            'cum_RMSE': {var:[] for var in variables}
        } for h in ["h=1", "h=3", "h=6", "h=12"]
    }

    for h in horizons:
        print(f"Running for horizon {h}...")
        # Create a copy to avoid modifying the original
        df = df_long.copy()

        # Split data into train and test
        split_idx = 180
        train_dates = sorted(df['ds'].unique())[:split_idx]

        train_data = df[df['ds'].isin(train_dates)]
        test_data = df[~df['ds'].isin(train_dates)]

        # Test train split for AR data (wide format)
        train_data_ar = df_processed.iloc[:split_idx]
        test_data_ar = df_processed.iloc[split_idx:]

        # Calculate RMSE and statistics for each variable
        current_cum_sums ={var:0 for var in variables}
        rmses = {var:[] for var in variables}  # Store all RMSE values for this horizon
        ar_rmses = {var:[] for var in variables}

        while split_idx < 245:
            # Generate forecast
            forecast_df = nixtla_client.forecast(
                df=train_data,
                h=h,
                level=[80, 90]
            )
            # Calculate metrics for each unique_id (variable)
            for var in df['unique_id'].unique():
                # Get actual values for this horizon and variable
                var_test = test_data[test_data['unique_id'] == var]['y'].iloc[:h].values
                # Get predictions for this variable
                var_pred = forecast_df[forecast_df['unique_id'] == var]['TimeGPT'].values

                # Calculate RMSE
                rmse = mean_squared_error(var_test, var_pred)
                ar1_rmse, _ = run_ar_benchmark(train_data_ar, test_data_ar, var, ar_order=1, horizon=h)

                # Track metrics
                current_cum_sums[var] += rmse
                results["h="+str(h)]['cum_RMSE'][var].append(current_cum_sums[var])
                rmses[var].append(rmse)
                ar_rmses[var].append(ar1_rmse)

            # Shift test-train split by 1 timestamp
            split_idx += 1
            train_dates = sorted(df['ds'].unique())[:split_idx]
            train_data = df[df['ds'].isin(train_dates)]
            test_data = df[~df['ds'].isin(train_dates)]

            train_data_ar = df_processed.iloc[:split_idx]
            test_data_ar = df_processed.iloc[split_idx:]

        for var in variables:
            # Compute root mean square error for each variable
            results["h="+str(h)]['RMSE'].append(np.sqrt((np.sum(rmses[var])/np.sum(ar_rmses[var]))))

    return results

# Example usage:
timegpt_results = run_timegpt(df_timegpt, df_processed)

# Print results
print("\nDetailed RMSE for each variable and horizon:")
print(timegpt_results)

Running for horizon 1...
Running for horizon 3...
Running for horizon 6...
Running for horizon 12...

Detailed RMSE for each variable and horizon:
{'h=1': {'RMSE': [np.float64(1.128790499936219), np.float64(1.1626042726806551), np.float64(1.0766531111076025), np.float64(1.0335791802009304), np.float64(0.9301995344186882), np.float64(0.978637833424465), np.float64(0.8367372291219783), np.float64(1.3823794109799796), np.float64(0.8197297880065321), np.float64(1.1126820382375673), np.float64(1.1924214247105647), np.float64(1.1675862668360892), np.float64(1.169485395004773), np.float64(0.997516501537385), np.float64(1.016536959036641), np.float64(1.0374254615354455), np.float64(1.1623665354994233), np.float64(1.0255166186877493), np.float64(0.9432303742155222), np.float64(0.9871950745243205)], 'cum_RMSE': {'GDP_log': [0.0001160698991983752, 0.00014747328577445978, 0.00019140644960201944, 0.00019199478616311547, 0.0002736504144921869, 0.0004607313314010644, 0.0005017520850616078, 0.00050988

In [ ]:
for h in timegpt_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(timegpt_results[h]['RMSE'])
    #rmse_array = rmse_array[rmse_array <= 5]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.0580636754851265
Std:  0.12882388952739893
Min:  0.8197297880065321
Max:  1.3823794109799796
Q25:  0.9850557642493567
Median:  1.035502320868188
Q75:  1.1624259697947312
IQR:  0.17737020554537453
Horizon h=3:
Mean: 1.0525078573446558
Std:  0.11990829070813412
Min:  0.7895446301336464
Max:  1.269720543609028
Q25:  0.9839003150562589
Median:  1.0261983838581537
Q75:  1.1574726656174021
IQR:  0.17357235056114328
Horizon h=6:
Mean: 1.0863081292506747
Std:  0.1388797598735204
Min:  0.9282194171075708
Max:  1.4056201707053553
Q25:  0.9622232707842432
Median:  1.0580851567662495
Q75:  1.1604240144516353
IQR:  0.19820074366739215
Horizon h=12:
Mean: 1.1455362122158235
Std:  0.1836655700553786
Min:  0.8988715646822438
Max:  1.6021600511839558
Q25:  1.0286504147997768
Median:  1.1596621121705284
Q75:  1.1907377015592515
IQR:  0.1620872867594747


In [ ]:
# Save to JSON
import json
with open('results/timegpt_results.json', 'w') as f:
    json.dump(timegpt_results, f,indent=4,)

# RAG Implementation

### Extract BEA Quarterly Reports for Economic Covariates

In [ ]:
import json

# Load your JSON file
with open("data/bea_report_data.json", "r") as f:
    bea_report_json = json.load(f)

In [14]:
import json
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

class BEARetriever:
    def __init__(self, bea_df, embed_cols, k=40):
        self.bea_df = bea_df.sort_values('ds')
        self.embed_cols = embed_cols
        self.embeddings = bea_df[embed_cols].values
        self.dates = pd.to_datetime(bea_df['ds'].values)
        self.k = k

    def get_context_embeddings_by_row(self, row_date):
        row_date = pd.to_datetime(row_date)
        valid_mask = self.dates <= row_date
        valid_embeddings = self.embeddings[valid_mask]

        if valid_embeddings.shape[0] == 0:
            return np.zeros(len(self.embed_cols))

        selected_embeddings = valid_embeddings[-self.k:] if valid_embeddings.shape[0] >= self.k else valid_embeddings
        return selected_embeddings.mean(axis=0)

def load_bea_sentiment_embeddings(bea_json_path, report):
    """Load BEA report texts and compute FinBERT sentiment vectors per report."""
    with open(bea_json_path, 'r') as f:
        bea_data = json.load(f)

    tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
    model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")
    model.eval()

    dates = []
    sentiment_vecs = []

    for date_str, report in bea_data[report].items():
        dates.append(pd.to_datetime(date_str))
        text = report['text']

        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1)
            sentiment_vec = probs[0].numpy()  # Shape: [neg, neu, pos]
            sentiment_vecs.append(sentiment_vec)

    bea_df = pd.DataFrame(sentiment_vecs, columns=['neg', 'neu', 'pos'])
    bea_df['ds'] = dates
    return bea_df, ['neg', 'neu', 'pos']


In [15]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
!pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.7 MB/s eta 0:00:00


In [ ]:
df_timegpt_gdp.loc[df_timegpt_gdp['unique_id'] == 'GPDI_log']

,ds,unique_id,y
774,1960-04-01,GPDI_log,-1.933337
775,1960-07-01,GPDI_log,-1.940230
776,1960-10-01,GPDI_log,-2.047059
777,1961-01-01,GPDI_log,-2.021036
778,1961-04-01,GPDI_log,-1.962365
...,...,...,...
1027,2023-07-01,GPDI_log,1.444928
1028,2023-10-01,GPDI_log,1.451383
1029,2024-01-01,GPDI_log,1.460634
1030,2024-04-01,GPDI_log,1.482571


In [16]:
bea_json_path = "data/bea_report_data.json"
bea_df, embed_cols = load_bea_sentiment_embeddings(bea_json_path, 'GDP')
gdp_retriever = BEARetriever(bea_df, embed_cols)

bea_df, embed_cols = load_bea_sentiment_embeddings(bea_json_path, 'International Transactions')
trade_retriever = BEARetriever(bea_df, embed_cols)

bea_df, embed_cols = load_bea_sentiment_embeddings(bea_json_path, 'Personal Income')
income_retriever = BEARetriever(bea_df, embed_cols)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [25]:
import time
def run_timegpt_rag(df_long, retrievers, ratio=0.7):
    variables = df_long['unique_id'].unique()

    horizons = [1, 3, 6, 12]
    results = {f"h={h}": {"RMSE": [], "rmses": {var: [] for var in variables}} for h in horizons}

    var_mappings = {
    'GDP': (['GDP_log', 'GPDI_log', 'INDPRO_log'], retrievers[0]),
    'International Transactions': (['XTIMVA01USM664S_log', 'XTEXVA01USM664S_log', 'ROWFDIQ027S','B235RC1Q027SBEA'], retrievers[1]),
    'Personal Consumption Expenditures': (['RPI_log', 'PCEPI_inflation', 'CPILFESL_inflation'], retrievers[2])
    }


    for h in horizons:
        print(f"Running for horizon {h}...")
        for report in var_mappings:
            print(f"Using {report} as context...")
            retriever = var_mappings[report][1]

            for var in var_mappings[report][0]:
                print(f"Using {var} as target variable...")
                df = df_long.loc[df_long['unique_id'] == var]

                split_idx = 180

                train_dates = sorted(df['ds'].unique())[:split_idx]
                train_data = df[df['ds'].isin(train_dates)].copy()
                test_data = df[~df['ds'].isin(train_dates)].copy()
                last_date = pd.to_datetime(train_dates[-1])

                rmses = {var: [] for var in variables}
                zero_rmses = {var: [] for var in variables}

                while split_idx < 245:
                    time.sleep(0.1)
                    contextual_embeddings = np.vstack([
                        retriever.get_context_embeddings_by_row(date)
                        for date in train_data['ds']
                    ])
                    embed_cols_dynamic = [f'pca_embed_{i}' for i in range(contextual_embeddings.shape[1])]
                    train_embed_df = pd.DataFrame(contextual_embeddings, columns=embed_cols_dynamic)
                    train_embed_df['ds'] = train_data['ds'].values

                    train_data['ds'] = pd.to_datetime(train_data['ds'])
                    train_embed_df['ds'] = pd.to_datetime(train_embed_df['ds'])

                    train_data_embed = pd.merge_asof(
                        train_data.sort_values('ds'),
                        train_embed_df.set_index('ds'),
                        left_on='ds',
                        right_index=True,
                        direction='nearest',
                    )

                    future_dates = pd.date_range(start=test_data['ds'].min(), periods=h, freq='Q')
                    future_embeds = np.vstack([
                        retriever.get_context_embeddings_by_row(date)
                        for date in future_dates
                    ])

                    X_future = pd.DataFrame(future_embeds, columns=embed_cols_dynamic)
                    X_future['ds'] = pd.to_datetime(future_dates)
                    X_future['unique_id'] = var  # only one target series

                    forecast_df = nixtla_client.forecast(
                        df=train_data_embed,
                        h=h,
                        X_df=X_future,
                        level=[80, 90]
                    )

                    zero_forecast_df = nixtla_client.forecast(
                        df=train_data,
                        h=h,
                        level=[80, 90]
                    )

                    var_test = test_data[test_data['unique_id'] == var]['y'].iloc[:h].values
                    var_pred = forecast_df[forecast_df['unique_id'] == var]['TimeGPT'].values

                    zero_var_pred = zero_forecast_df[zero_forecast_df['unique_id'] == var]['TimeGPT'].values

                    rmse = mean_squared_error(var_test, var_pred)
                    zero_rmse = mean_squared_error(var_test, zero_var_pred)
                    results[f"h={h}"]['rmses'][var].append(rmse)
                    rmses[var].append(rmse)

                    zero_rmses[var].append(zero_rmse)

                    split_idx += 1
                    train_dates = sorted(df['ds'].unique())[:split_idx]
                    train_data = df[df['ds'].isin(train_dates)].copy()
                    test_data = df[~df['ds'].isin(train_dates)].copy()
                    last_date = pd.to_datetime(train_dates[-1])

                ratio = np.sqrt((np.sum(rmses[var]) / np.sum(zero_rmses[var])))
                print(f"Ratio for {var}: {ratio}")
                results[f"h={h}"]['RMSE'].append(ratio)
                print(f"RMSEs for {var}: ", results[f"h={h}"]["rmses"][var])

    return results


#df_timegpt_gdp = df_timegpt.loc[(df_timegpt['unique_id'] == 'GDP_log') | (df_timegpt['unique_id'] == 'GPDI_log')]
retrievers = [gdp_retriever, trade_retriever, income_retriever]
timegpt_rag_results = run_timegpt_rag(df_timegpt, retrievers)

Running for horizon 1...
Using GDP as context...
Using GDP_log as target variable...
Ratio for GDP_log: 1.4248955696789105
RMSEs for GDP_log:  [6.857006048827428e-10, 4.566877365425325e-05, 7.03696216283093e-05, 0.000281702622297756, 6.012135431889343e-07, 9.24274072664882e-05, 0.006060810178959406, 0.007611139383685491, 5.6852696426176124e-05, 7.741387641569724e-07, 3.568894536399261e-05, 1.6355688894892847e-05, 1.8149569820948175e-06, 3.0194507817694138e-05, 0.0005653485631321407, 0.0005233251882344714, 9.718482660515745e-05, 8.107161925686811e-06, 5.898234179470935e-05, 9.620423963881252e-07, 0.00016625376900192618, 5.803753488510686e-05, 0.00015266759799222341, 3.3709579634869542e-06, 9.175158707349043e-05, 1.4941750850142185e-05, 0.00018604346684512684, 0.00023729143040485297, 8.127989415329855e-05, 4.53833152988476e-06, 1.2189337993756138e-07, 1.2907001663583336e-05, 9.344284210047409e-06, 2.492825628625614e-05, 3.060359057995918e-05, 4.4739069125565575e-05, 3.6989168043036735e-0

ERROR:nixtla.nixtla_client:Attempt 1 failed with error: status_code: 502, body: Could not parse JSON: b'Missing request, possibly due to expiry or cancellation'


Ratio for XTIMVA01USM664S_log: 1.560931438124738
RMSEs for XTIMVA01USM664S_log:  [0.04261476224187055, 0.011585309322766158, 0.0066893303546142376, 0.0005072429381369294, 0.0026801702221376737, 0.0009237449292498334, 0.0005452810718750419, 0.008133761413733521, 0.01413052385938667, 0.01795257825536136, 0.020357384931620533, 0.02580270262821764, 0.027250415964355276, 0.027344131875179367, 0.005327836500209959, 0.006616502267320925, 0.02695577507588767, 0.013532844192982193, 0.011467422626890844, 0.010597376579933036, 0.00868517355040092, 0.009803484577012302, 0.011559151084915209, 0.009606886642125055, 0.012844408180718725, 0.006756994761009161, 0.03035761673009572, 0.01436788954423561, 0.002701756610432081, 0.0019456573050236298, 0.004145757457976794, 0.0032483351258568646, 0.002336741576108198, 0.0015805478410412385, 0.001526472649884924, 0.0004089300046766367, 0.007607186785992572, 0.015315329210317238, 0.03508309287332846, 0.04695655533509837, 0.033364902248836766, 0.071582310158833

ERROR:nixtla.nixtla_client:Attempt 1 failed with error: status_code: 502, body: Could not parse JSON: b'Missing request, possibly due to expiry or cancellation'
ERROR:nixtla.nixtla_client:Attempt 2 failed with error: status_code: 502, body: Could not parse JSON: b'Missing request, possibly due to expiry or cancellation'


Ratio for XTEXVA01USM664S_log: 1.3357643217115718
RMSEs for XTEXVA01USM664S_log:  [0.01727618964231249, 0.0014789259692781144, 0.0049637933100209666, 0.0016722690598952653, 0.0004614231090071313, 0.00240488241931802, 0.003852429038206817, 0.013387257137644654, 0.01848919041153385, 0.017621898937481482, 0.019549501828552435, 0.026578242560151902, 0.022158050369573593, 0.027675158851771797, 0.003569196249068511, 0.004096788441210356, 0.014448398566758357, 0.013404069706297263, 0.013063803467840153, 0.012117648114085519, 0.012861621608528473, 0.013770986399428859, 0.014588772607798742, 0.010004889881182442, 0.008917098236931969, 0.0026347897275193106, 0.04170188151439958, 0.015790259333090733, 0.00814877459037725, 0.003723469346387886, 0.005808532569713792, 0.004892417447600835, 0.0118446588807219, 0.02287474020855512, 0.00545286889814675, 0.001531205238604218, 0.0047952878691085166, 0.0030922226180048356, 0.03097194574836254, 0.03752976025237243, 0.006080191183687369, 0.06220377201868795

In [26]:
for h in timegpt_rag_results:
    print(f"Horizon {h}:")
    rmse_array = np.array(timegpt_rag_results[h]['RMSE'])
    rmse_array = rmse_array[rmse_array <= 5]
    # Calculate statistics for this horizon
    print("Mean:", np.mean(rmse_array))
    print("Std: ", np.std(rmse_array))
    print("Min: ", np.min(rmse_array))
    print("Max: ", np.max(rmse_array))
    print("Q25: ", np.percentile(rmse_array, 25))
    print("Median: ", np.percentile(rmse_array, 50))
    print("Q75: ", np.percentile(rmse_array, 75))
    print("IQR: ", np.percentile(rmse_array, 75) - np.percentile(rmse_array, 25))

Horizon h=1:
Mean: 1.4045050299314012
Std:  0.5025892850113978
Min:  0.9373365887507232
Max:  2.6263352874903316
Q25:  1.0264967006565595
Median:  1.2705292154203218
Q75:  1.4202743718591402
IQR:  0.39377767120258067
Horizon h=3:
Mean: 1.329260653565656
Std:  0.3257308118927829
Min:  0.9633163001805614
Max:  1.868995265098037
Q25:  1.0293056785917822
Median:  1.219615171477523
Q75:  1.57341473149445
IQR:  0.5441090529026678
Horizon h=6:
Mean: 1.319874814946016
Std:  0.32790233477494135
Min:  0.9705204520697633
Max:  1.939896386564553
Q25:  1.0565314106537955
Median:  1.1905522478918256
Q75:  1.5046396590214464
IQR:  0.44810824836765084
Horizon h=12:
Mean: 1.287618419993478
Std:  0.32457269480176676
Min:  0.9722538412028484
Max:  1.9842513714071695
Q25:  1.0286720233921376
Median:  1.183668039851744
Q75:  1.4614656096461243
IQR:  0.4327935862539867


In [27]:
# Save to JSON
import json
with open('results/rag_forecasting/timegpt_rag_results.json', 'w') as f:
    json.dump(timegpt_rag_results, f,indent=4,)